In [4]:
import napari
import numpy as np
import tifffile as tiff
import zarr
import dask.array as da
from dask.diagnostics import ProgressBar
import pandas as pd
import xml.etree.ElementTree as ET
import seaborn as sns
import matplotlib.pyplot as plt

In [1]:
image_path = '/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/cycif_ovca/img/S012_chemonaive.ome.tif'

In [2]:
def print_metadata_structure(element, indent=0):
    print("  " * indent + f"Tag: {element.tag}")
    if element.attrib:
        print("  " * (indent + 1) + "Attributes:")
        for key, value in element.attrib.items():
            print("  " * (indent + 2) + f"{key}: {value}")
    for child in element:
        print_metadata_structure(child, indent + 1)

In [5]:
def open_image_napari(image_path, markers:list) -> napari.Viewer:
    # create a colormap list with the same length as markers - ['cyan', 'green', 'yellow', 'red', 'magenta', 'orange', 'blue' ] repeat until reach len(markers)
    colormap = ['cyan', 'green', 'yellow', 'red', 'magenta', 'orange', 'blue']
    colormap = colormap * (len(markers) // len(colormap) + 1)

    store = tiff.imread(image_path, aszarr=True)
    cache = zarr.LRUStoreCache(store, max_size=2**30)
    zobj = zarr.open(cache, mode='r')
    data = da.from_array(zobj)
    n_channels = data.shape[0]
    viewer = napari.Viewer()
    # Add each channel as a separate image layer
    for i in range(10):
        channel_data = [data[i]]  # Extract the i-th channel
        viewer.add_image(
            channel_data,
            name=markers[i],
            rgb=False,
            contrast_limits=[0, 2**12],
            blending='additive',
            colormap=colormap[i],
        )
    napari.run()
    store.close()

    return viewer

In [ ]:
store = tiff.imread(image_path, aszarr=True)
cache = zarr.LRUStoreCache(store, max_size=2**30)
zobj = zarr.open(cache, mode='r')

In [ ]:
list(zobj.attrs)

In [ ]:
zobj

In [ ]:


dask_array = da.from_zarr(zobj)

In [ ]:
dask_array[0]

In [ ]:
with tiff.TiffFile(image_path) as tif:
    ome_metadata = tif.ome_metadata
    
    if ome_metadata:
        root = ET.fromstring(ome_metadata)
        print("Complete OME Metadata Structure:")
        print_metadata_structure(root)
    else:
        print("No OME metadata found in the file.")

In [6]:
with tiff.TiffFile(image_path) as tif:
    md = tif.ome_metadata
    root = ET.fromstring(md)
    ns = {'ome': 'http://www.openmicroscopy.org/Schemas/OME/2016-06'}
    channels = root.findall('ome:Image/ome:Pixels/ome:Channel', ns)
    channel_names = [channel.get('Name') for channel in channels]


In [7]:
channel_names

['Channel 0',
 'Channel 1',
 'Channel 2',
 'Channel 3',
 'Channel 4',
 'Channel 5',
 'Channel 6',
 'Channel 7',
 'Channel 8',
 'Channel 9',
 'Channel 10',
 'Channel 11',
 'Channel 12',
 'Channel 13',
 'Channel 14',
 'Channel 15',
 'Channel 16',
 'Channel 17',
 'Channel 18',
 'Channel 19',
 'Channel 20',
 'Channel 21',
 'Channel 22',
 'Channel 23',
 'Channel 24',
 'Channel 25',
 'Channel 26',
 'Channel 27',
 'Channel 28',
 'Channel 29',
 'Channel 30',
 'Channel 31',
 'Channel 32',
 'Channel 33',
 'Channel 34',
 'Channel 35',
 'Channel 36',
 'Channel 37',
 'Channel 38',
 'Channel 39',
 'Channel 40',
 'Channel 41',
 'Channel 42',
 'Channel 43',
 'Channel 44',
 'Channel 45',
 'Channel 46',
 'Channel 47',
 'Channel 48',
 'Channel 49',
 'Channel 50',
 'Channel 51',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
with tiff.TiffFile(image_path) as tif:
    md = tif.ome_metadata
    root = ET.fromstring(md)
    ns = {'ome': 'http://www.openmicroscopy.org/Schemas/OME/2016-06'}
    channels = root.findall('ome:Image/ome:Pixels/ome:Channel', ns)
    channel_names = [channel.get('Name') for channel in channels]    
    with open('/Users/lukashat/Documents/PhD_Schapiro/Projects/TSPC/TSPC_NSCLC/antibody_validation/2025_03/markers.csv', 'w') as f:
            f.write('cycle,marker_name\n')
            for name in channel_names:
                f.write(f"1,{name}\n")

In [8]:
open_image_napari(image_path, channel_names)

/Users/lukashat/miniforge3/envs/napari-viewer/lib/python3.10/site-packages/napari/_vispy/layers/scalar_field.py:195: UserWarning: data shape (15888, 35716) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/Users/lukashat/miniforge3/envs/napari-viewer/lib/python3.10/site-packages/napari/_vispy/layers/scalar_field.py:195: UserWarning: data shape (15888, 35716) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/Users/lukashat/miniforge3/envs/napari-viewer/lib/python3.10/site-packages/napari/_vispy/layers/scalar_field.py:195: UserWarning: data shape (15888, 35716) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/Users/lukashat/miniforge3/envs/napari-viewer/lib/python3.10/site-packages/napari/_vispy/layers/scalar_field.py:195: UserWarning: data s

Viewer(camera=Camera(center=(0.0, np.float64(7943.5), np.float64(17857.5)), zoom=np.float64(0.03598807257251652), angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True, orientation=(<DepthAxisOrientation.TOWARDS: 'towards'>, <VerticalAxisOrientation.DOWN: 'down'>, <HorizontalAxisOrientation.RIGHT: 'right'>)), cursor=Cursor(position=(1.0, 1.0), scaled=True, style=<CursorStyle.STANDARD: 'standard'>, size=1.0), dims=Dims(ndim=2, ndisplay=2, order=(0, 1), axis_labels=('0', '1'), rollable=(True, True), range=(RangeTuple(start=np.float64(0.0), stop=np.float64(15887.0), step=np.float64(1.0)), RangeTuple(start=np.float64(0.0), stop=np.float64(35715.0), step=np.float64(1.0))), margin_left=(0.0, 0.0), margin_right=(0.0, 0.0), point=(np.float64(7943.0), np.float64(17857.0)), last_used=0), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False, spacing=0.0), layers=[<Image layer 'Channel 0' at 0x31b5caad0>, <Image layer 'Channel 1' at 0x31e25e4d0>, <Image layer 'Channel 2' at 

/Users/lukashat/miniforge3/envs/napari-viewer/lib/python3.10/site-packages/napari/_vispy/layers/scalar_field.py:195: UserWarning: data shape (15888, 35716) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(


In [ ]:
with tiff.TiffFile(image_path) as tif:
    zarr_store = tif.aszarr()
    zarr_group = zarr.open(zarr_store, mode='r')
    datasets = zarr_group.attrs['multiscales'][0]['datasets']
    dask_arrays = [
        da.array.from_zarr(zarr_store, component=ds['path'])
        for ds in datasets
    ]
    fully_res_array= dask_arrays[0]

In [ ]:
fully_res_array[:,:,:]

In [ ]:
intensities = fully_res_array[2,:,:]
min_val = intensities.min().compute()
max_val = intensities.max().compute()

In [ ]:
hist_vals, bin_edges = da.array.histogram(intensities, bins=100, range=(min_val, max_val), density=True)
hist_vals, bin_edges = da.array.compute(hist_vals, bin_edges)

In [ ]:
plt.step(bin_edges[:-1], hist_vals, where='post')
plt.yscale('log')
plt.xlabel('Intensity_values (12bit exported)')
plt.ylabel('Frequency normed')

In [ ]:
max_val